In [ ]:
from math import *
import numpy as np
import pandas as pd
from glob import glob
import pickle
from scipy.interpolate import CubicSpline
import time
import matplotlib.pyplot as plt

In [ ]:
directory = '/home/ariya/Desktop/github/Airfoil_Polar_prediction/aiaa_conference/xf/runs/'

In [ ]:
# get the folders
af_files = [i.replace(directory, '') for i in glob(directory+'*')]
files_dic, afs = {}, {}
for i in range(len(af_files)):
    af = af_files[i]
    dir_name = '{}{}/'.format(directory,af)
    afs[af] = [j for j in glob(dir_name+'*.af')]
    files_dic[af] = [j for j in glob(dir_name+'*.polar')]

# get the interpolation points -> predefined points
#with open('interp.pickle', 'rb') as fid:
#    interp = pickle.load(fid, encoding='latin1')
interp = np.linspace(0, 1., 162).reshape(1,-1)

In [ ]:
# get the airfoils (optimized from 14s to 500ms)
# use the interp to interpolate on x make data a series in y
afs_passed = []
af_data_dic = {}
for af in af_files:
    for files in afs[af]:
        af_name = af
        if af_name not in afs_passed:
            afs_passed.append(af_name)
            
            with open(files,'r') as fid:
                af_data = fid.readlines()
                
            af_name = af_data[0].replace('\r\n', '').replace(' ', '').lower()
            x = []
            y = []
            for lines in range(1, len(af_data)):
                line = af_data[lines].replace('\r\n', '').split()
                x.append(float(line[0]))
                y.append(float(line[1]))

            x = np.array(x).reshape(1,-1)
            y = np.array(y).reshape(1,-1)

            # split the upper and lower surfaces
            x_min = x.argmin()
            x_up = x[0,0:x_min+1]
            x_do = x[0,x_min:]
            y_up = y[0,0:x_min+1]
            y_do = y[0,x_min:]
            
            # remove x duplicates
            x_up, index = np.unique(x_up, axis=0, return_index=True)
            y_up = y_up[index].copy()
            x_do, index = np.unique(x_do, axis=0, return_index=True)
            y_do = y_do[index].copy()
            
            # get the interpolation points
            int_min = interp.argmin()
            x_up_int = interp
            x_do_int = interp
            # interpolate
            #y_up_int = np.flip(np.interp(np.flip(x_up_int,0), np.flip(x_up, 0), np.flip(y_up,0)), 0)
            #y_do_int = np.interp(x_do_int, x_do, y_do)
            
            y_up_int = np.flip(CubicSpline(x_up, y_up)(np.flip(x_up_int,0)), 0)
            y_do_int = CubicSpline(x_do, y_do)(x_do_int)
            
            nn_input = np.concatenate((y_up_int.reshape(1,-1),np.flip(y_do_int).reshape(1,-1)), axis=1)
            
            af_data_dic[af_name] = {}
            af_data_dic[af_name]['x'] = np.array(x).reshape(1,-1)
            af_data_dic[af_name]['y'] = np.array(y).reshape(1,-1)
            af_data_dic[af_name]['x_up'] = x_up.reshape(1,-1)
            af_data_dic[af_name]['x_do'] = x_do.reshape(1,-1)
            af_data_dic[af_name]['y_up'] = y_up.reshape(1,-1)
            af_data_dic[af_name]['y_do'] = y_do.reshape(1,-1)
            af_data_dic[af_name]['x_up_int'] = x_up_int.reshape(1,-1)
            af_data_dic[af_name]['x_do_int'] = x_do_int.reshape(1,-1)
            af_data_dic[af_name]['y_up_int'] = y_up_int.reshape(1,-1)
            af_data_dic[af_name]['y_do_int'] = y_do_int.reshape(1,-1)
            af_data_dic[af_name]['input'] = nn_input

# save the results
with open('tr_af_points.pickle', 'wb') as fid:
    pickle.dump(af_data_dic, fid)

In [ ]:
# enumerate afs
af_label = {name: num for num, name in enumerate(af_data_dic.keys())}
label_af = {}
for i in af_label.keys():
    label_af[af_label[i]] = i
with open('tr_af_labels.pickle', 'wb') as fid:
    pickle.dump(af_label, fid)
with open('tr_label_afs.pickle', 'wb') as fid:
    pickle.dump(label_af, fid)

In [ ]:
%%time
# get the polars data for each airfoil - 1081673 many data currently
polars = []
nn_input = []
for af in af_files:
    for files in files_dic[af]:
        try:
            with open(files, 'r') as fid:
                polar = fid.readlines()
            af_n = af_label[polar[3].replace('Calculated polar for:','').replace(' ', '').
                          replace('\r\n', '').lower()]
            re = float(files.split('/')[-1].split('_')[-1].replace('.polar', ''))

            index = 0
            for lines in range(len(polar)):
                line = polar[lines].split()
                try:
                    if line[0] == 'alpha':
                        index = lines
                except:
                    continue
            for k in range(index+2, len(polar)):
                l = polar[k].replace('\r\n', '').split()
                cur_list = [af_n, re]
                ll = [float(m) for m in l[:5]]
                cur_list.extend(ll)
                polars.append(cur_list)
        except:
            print(af)

In [ ]:
# save, both as a dataframe and a pickle
a = pd.DataFrame(polars, columns=['af', 're', 'a', 'cl', 'cd', 'cdp', 'cm'])
a.to_csv('tr_raw_af_data.txt', index=False)

In [ ]:
# ignore it
with open('tr_polars.pickle', 'w') as fid:
    pickle.dump(polars, fid)